# controll the robot

## import libraries


In [ ]:
import cv2
import os
import random
import numpy as np
import tensorflow.lite as tflite


We need the .tflite file to run the model on the robot. So we start by importing the model. It is in the same folder as the notebook.

In [ ]:
tflite_model_filename = "mini_proj_model_v01_01.tflite"

In [ ]:
interpreter = tflite.Interpreter(tflite_model_filename)
interpreter.allocate_tensors()

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

In [ ]:
print(input_shape)

## car setup

#### Initialize the car and camera classes

Create the racecar class

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

Create the camera class.

In [ ]:
from jetcam.csi_camera import CSICamera
camera = CSICamera(width=224, height=224, capture_fps=65)

setup the gains and bias for the drive function

In [ ]:
STEERING_GAIN = 0.6
STEERING_BIAS = 0.00
THROTTLE_GAIN = 0.8

In [ ]:
def preprocess(image):
    assert image.shape == (224,224,3)
    # crop the top 80 pixels
    image = image[80:,:,:]
    # check the input shape of the model and if the last dimension is 1, then convert to grayscale
    if input_shape[3] == 1:
        # convert to grayscale
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # resize to the input shape of the model
    image = cv2.resize(image, (input_shape[2], input_shape[1]))
    # Scale the images to the range of [0, 1]
    image = image / 255.0
    return image

## driving the car (fully autonomous)

In [ ]:
try:
    # main loop
    while True:
        image = camera.read()
        image = preprocess(image)
        # add a batch dimension
        image = np.expand_dims(image, axis=0)
        # set the input tensor
        interpreter.set_tensor(input_details[0]['index'], image)
        # run the inference
        interpreter.invoke()
        # get the output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])
        # get the steering and throttle values
        steering = output_data[0][0]
        throttle = output_data[0][1]
        # set the car controls
        car.steering = steering
        car.throttle = throttle

except:
    car.throttle = 0.0
    raise

## driving the car (semi-autonomous - manual throttle)

In [ ]:
try:
    # main loop
    while True:
        image = camera.read()
        image = preprocess(image)
        # add a batch dimension
        image = np.expand_dims(image, axis=0)
        # set the input tensor
        interpreter.set_tensor(input_details[0]['index'], image)
        # run the inference
        interpreter.invoke()
        # get the output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])
        # get the steering and throttle values
        steering = output_data[0][0]
        throttle = output_data[0][1]
        # set the car controls
        car.steering = steering
        car.throttle = throttle

except:
    car.throttle = 0.0
    raise

## E stop

In [ ]:
car.throttle = 0.0